In [1]:
import rasterio
import pandas as pd
import os,glob
from rasterstats import zonal_stats
from tkinter import filedialog
from tkinter import *
import geopandas as gpd
from datetime import datetime,timedelta
import exiftool
import numpy as np
import shutil
from tqdm import tqdm

In [2]:
import micasense.metadata as metadata
import micasense.imageset as imageset
import micasense.utils as msutils
import micasense.plotutils as plotutils 
import micasense.capture as capture
import micasense.dls as dls
from micasense.panel import Panel
from micasense.image import Image

from PIL import Image as Img
import numpy as np

import math
import matplotlib.pyplot as plt
import subprocess
import pyzbar.pyzbar as pyzbar

In [26]:
#### 2020 için #####


excel = r"D:\pytseb\\uav-metric\Eto-Etr_v3.xlsx"
excel = pd.read_excel(excel, 'saatlik')


ETr_h   =excel.iloc[8:9,1:41].values.tolist()
ETr_d   =excel.iloc[9:10,1:41].values.tolist()
u       =excel.iloc[4:5,1:41].values.tolist()
hava_sıc=excel.iloc[3:4,1:41].values.tolist()





#### 2021 için #####

#excel = r"D:\pytseb\uav-metric\Eto-Etr_v2.xlsx"
#excel = pd.read_excel(excel, 'saatlik')


#ETr_h   =excel.iloc[19:20,1:32].values.tolist()
#ETr_d   =excel.iloc[20:21,1:32].values.tolist()
#u       =excel.iloc[15:16,1:32].values.tolist()
#hava_sıc=excel.iloc[14:15,1:32].values.tolist()



In [27]:
kayitYeri= r"D:\pytseb\uav-metric\METRIC\2020"

klasor = r"J:\2021_TUBITAK_sorgum\2020v2\ham\raw"
klasor2= r"J:\2021_TUBITAK_sorgum\2020v2\ham\UAV_Albedo"
klasor3= r"J:\2021_TUBITAK_sorgum\2020sorgum\Altum_2020_paket"
stack_= []
albedo_=[]
dem_=[]
panel=[]
panel2_=[]

for root, dirname, files in os.walk(klasor):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-4:]=='.tif':
            stack_.append(isim)

for root, dirname, files in os.walk(klasor2):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-10:]=='albedo.tif':
            albedo_.append(isim)

for root, dirname, files in os.walk(klasor3):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-12:]=='DEM_02cm.tif':
            dem_.append(isim)
            
for root, dirname, files in os.walk(klasor3):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-14:-9]=='IMG_0':
            panel.append(isim)          
            

for i in range(len(panel)):
    #print(i)
    if i%5==0:
        panel2_.append(panel[i])    


    
            
print(len(stack_))
print(len(albedo_))
print(len(dem_))
print(len(panel2_))

37
37
37
37


In [6]:
#bu kod programın çalıştığı yerde exiftools yok ise çalıştırılacak
os.chdir("D:\pytseb")

In [28]:
for i in range(len(stack_)): # len görüntü sayısı kadar değiştirilcek
    
    
    Ta = hava_sıc[0][i] + 273.16  
    ux= u[0][i]           
    ETr_h_ = ETr_h[0][i]
    ETr_d_ = ETr_d[0][i]
    
    print("Hava Sıcaklığı: ", Ta)
    print("Rüzgar Hızı: ", ux)

    #blue    = rasterio.open(stack_[0], mode="r").read(1)
    #green   = rasterio.open(stack_[0], mode="r").read(2)
    red     = rasterio.open(stack_[i], mode="r").read(3)
    #rededge = rasterio.open(stack_[i], mode="r").read(4)
    NIR     = rasterio.open(stack_[i], mode="r").read(5)
    ts      = ((rasterio.open(stack_[i], mode="r").read(6))*0.8038 +4.0485) + 273.16
    albedo  = rasterio.open(albedo_[i], mode="r").read(1)
    dem     = rasterio.open(dem_[i], mode="r").read(1)
    #wp_image= rasterio.open(panel2[i], mode="r").read(1)
    
    
    


    kwargs = rasterio.open(stack_[i], mode="r").meta
    kwargs.update(dtype=rasterio.float32, count=1)
    
    head, tail = os.path.split(stack_[i]) #dosya adını çekiyor


    print("Görüntüler İçeri Aktarıldı")
    
    


    #################################################################################################################
    #################################################################################################################
    #Rs-Gelen Hesabı (DEM)


    Gsc=1367
    with exiftool.ExifTool() as et:
        #GPSLongitude = et.get_tag('EXIF:GPSLongitude',path )
        SolarElevation = float(et.get_tag('XMP:SolarElevation',panel2_[i]))  #radyan olarak veriyor, kullanıma hazır
        print("SolarElevation: ", SolarElevation)
        #SolarElevation = float(et.get_tag('XMP:SolarElevation',wp_image[0]))

        str_time = et.get_tag('EXIF:DateTimeOriginal',panel2_[i] )
        #str_time = et.get_tag('EXIF:DateTimeOriginal',wp_image[0] )
        j =(datetime.strptime(str_time, "%Y:%m:%d %H:%M:%S").strftime('%j'))   #doy hesabı
        print("DOY: ",j)

    j   = float(j)
    des = 1 + 0.033 * np.cos(j * 2* float(np.pi)/365)   
    Tsw = 0.75 + 2*10**-5 * dem  

    Rs_gelen = Gsc * np.sin(SolarElevation) * des * Tsw
    
    #################################################################################################################
    #################################################################################################################
    #RL-Giden Hesabı (NDVI[arraylerden hesaplandı, import edilmese de olabilir mi?, kontrol et], Ts)
    NDVI      = (NIR-red)/(NIR+red)
    SB_sabiti = 0.0000000567
    Eo        = 1.009 + 0.047 * np.log(NDVI)

    RL_Giden  = SB_sabiti * Eo * (ts)**4

    #################################################################################################################
    #################################################################################################################
    #RL-Gelen Hesabı(Ta(K))

    #B1 = 0.94  #orjinal
    B1 = 0.95
    #B2 = 0.11  #orjinal
    B2 = 0.09
    #Ea = B1 * (-np.exp(Tsw))**B2
    Ea = B1 * np.array((-np.log(Tsw)+0j))**B2
    Rl_gelen = SB_sabiti * Ea * Ta**4

    #################################################################################################################
    #################################################################################################################
    #Rn-Hesabı

    Rn = (1-albedo)* Rs_gelen + (Rl_gelen - RL_Giden - (1 - Eo) * Rl_gelen)
    Rns = (1-albedo)* Rs_gelen
    Rnl = (Rl_gelen - RL_Giden - (1 - Eo) * Rl_gelen)
    Rn= np.where(Rn>0, Rn, 0)

    with rasterio.open(kayitYeri+"/Rn/"+str(tail[:8])+'_Rn.tif', 'w', **kwargs) as dst:
        dst.write_band(1, Rn.astype(rasterio.float32))
    print("Rn Görüntüsü Oluşturuldu")
    
#################################################################################################################
    #G Hesabı    
    
    
    
    
    G= np.where(NDVI>0, (Rn*((ts-273.15)/albedo) * (0.0035* albedo + 0.0072* albedo**2) * (1-0.98*NDVI**4)), 0.5)
    G= np.where(G>0, G, 0)

    with rasterio.open(kayitYeri+"/G/"+str((tail[:8]))+'_G.tif', 'w', **kwargs) as dst:
        dst.write_band(1, G.astype(rasterio.float32))
    print("G Görüntüsü oluşturuldu")
    
    #################################################################################################################
    #H Hesabı 


    R=287
    cP=1004
    z2=2
    z1=0.1

    k=0.41

    B5 = -5.51
    B6 = 5.8

    zom = np.exp(B5 + B6 * NDVI)
    #zom = 0.12 * dem    #METRIC

    zom_star = 0.123 * 0.15

    u100 = ux * np.log(100/zom_star)/np.log(2/zom_star)
    print("u100: ", u100)

    ustar = k * u100 / np.log(100/zom)

    Rah = np.log(z2/z1) / (ustar * k) # k=0.41

    P = 101.3 * ((293-0.0065 * dem)/293)**5.26

    q = 1000 * P / (1.01 * (ts-(ts-Ta)) * R)  # ro

    H= q * cP * (ts-Ta) / Rah
    H= np.where(H>0, H, 0)

    with rasterio.open(kayitYeri+"/H/"+str((tail[:8]))+'_H-ilk.tif', 'w', **kwargs) as dst:
        dst.write_band(1, H.astype(rasterio.float32))
        print("H ilk Görüntüsü Oluşturuldu")



    L1      = (-q * cP * ustar**3 * ts)/ (k * 9.81 * H)


    lst_mask = np.amin(ts[NDVI>0.8])
    print("LST MASK Değeri: ", lst_mask)
    mask = tuple(np.argwhere(lst_mask == ts)[0])  #lst yerine L lar gelecek
    katsayi=1
    iterasyon_sayisi=1

    while katsayi>0.15:


        H_ilk = H




        x100_1  = (1-16*100/L1)**(1/4)
        x2_1    = (1-16*2/L1)**(1/4)
        x01_1   = (1-16*0.1/L1)**(1/4)
        Ym100_1 = np.where(L1<0, (2 * np.log((1+x100_1)/2) +(np.log((1+x100_1**2)/2)) - (2 * np.arctan(x100_1))+ 0.5 * np.pi),-9999)
        Yh2_1   = np.where(L1<0, (2 * np.log((1+x2_1**2)/2)),-9999)
        Yh01_1  = np.where(L1<0, (2 * np.log((1+x01_1**2)/2)),-9999)

        ustar_1 = (k * u100) / (np.log(100/zom)-  Ym100_1)      
        Rah1    = (np.log(z2/z1)-Yh2_1+Yh01_1)/ (ustar_1 * k) # k=0.41



        #2.iterasyon başlar
        H      = q * cP * (ts-Ta) / Rah1

        L1      = (-q * cP * ustar_1**3 * ts)/ (k * 9.81 * H)





        katsayı = np.absolute(H[mask] - H_ilk[mask]) / np.absolute(H_ilk[mask])


        if katsayı>0.15:
            print("{}. İterasyon Tamamlandı" .format(iterasyon_sayisi))

            print("Katsayı Değeri: ", katsayı)
            with rasterio.open(kayitYeri+"/H/"+str((tail[:8]))+'_H'+str(iterasyon_sayisi)+'.tif', 'w', **kwargs) as dst:
                dst.write_band(1, H.astype(rasterio.float32))
            print("H{} Görüntüsü oluşturuldu".format(iterasyon_sayisi))
            iterasyon_sayisi = iterasyon_sayisi+1

        if katsayı<0.15: 

            print("{}. İterasyonda Çözüm Bulundu" .format(iterasyon_sayisi))
            print("Katsayı Değeri: ", katsayı)

            with rasterio.open(kayitYeri+"/H/"+str((tail[:8]))+'_H'+str(iterasyon_sayisi)+'.tif', 'w', **kwargs) as dst:
                dst.write_band(1, H.astype(rasterio.float32))
            print("H{} Görüntüsü oluşturuldu".format(iterasyon_sayisi))
            break
    #################################################################################################################
#################################################################################################################
    #LE Hesabı 

    LE = Rn - G - H
    LE= np.where(LE>0, LE, 0)

    with rasterio.open(kayitYeri+"/LE/"+str((tail[:8]))+'_LE.tif', 'w', **kwargs) as dst:
        dst.write_band(1, LE.astype(rasterio.float32))
    print("LE Görüntüsü oluşturuldu")
    
    #################################################################################################################
#################################################################################################################
    #LE to ETd Hesabı 
    
    
    
    lamda  = (2.501- 0.00236* (ts-273.15))*10**6
    ETinst = 3600 * LE / lamda
    ETrF = ETinst / ETr_h_
    ETd  = ETrF * ETr_d_
    ETd  = np.where(ETd<0,0,ETd)

    with rasterio.open(kayitYeri+"/ETd/"+str((tail[:8]))+'_ET_daily.tif', 'w', **kwargs) as dst:
        dst.write_band(1, ETd.astype(rasterio.float32))
    print("ETd Görüntüsü oluşturuldu")

    with rasterio.open(kayitYeri+"/ETrF/"+str((tail[:8]))+'_ETrF_daily.tif', 'w', **kwargs) as dst:
        dst.write_band(1, ETrF.astype(rasterio.float32))
    print("ETrF Görüntüsü oluşturuldu")
    
    
    
    
    
    


Hava Sıcaklığı:  305.76000000000005
Rüzgar Hızı:  3.8
Görüntüler İçeri Aktarıldı
SolarElevation:  0.8945894645144542
DOY:  136


C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in true_divide
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in log
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in multiply
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in subtract
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in multiply
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in subtract
C:\Users\CBS-3\anaconda3\envs\micasense\

Rn Görüntüsü Oluşturuldu


C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:104: ComplexWarning: Casting complex values to real discards the imaginary part


G Görüntüsü oluşturuldu
u100:  6.972471420928504
H ilk Görüntüsü Oluşturuldu


...

ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  291.66
Rüzgar Hızı:  4
Görüntüler İçeri Aktarıldı
SolarElevation:  1.068170694166338
DOY:  147
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  7.339443600977372
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  290.68997
1. İterasyon Tamamlandı
Katsayı Değeri:  inf
H1 Görüntüsü oluşturuldu
2. İterasyon Tamamlandı
Katsayı Değeri:  0.9994967
H2 Görüntüsü oluşturuldu
3. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.0
H3 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  295.16
Rüzgar Hızı:  2.15
Görüntüler İçeri Aktarıldı
SolarElevation:  1.230376941140605
DOY:  153
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  3.9449509355253376
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  299.43625
1. İterasyon Tamamlandı
Katsayı Değeri:  1.404132
H1 Görüntüsü oluşturuldu
2. İterasyon Tamamlandı
Katsayı Değeri:  0.28156903
H2 Görüntüsü oluşturuldu
3. İtera

C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\CBS-3\anaconda3\envs\micasense\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in multiply


G Görüntüsü oluşturuldu
u100:  8.256874051099544
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  297.8795
1. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.07240527
H1 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  299.66
Rüzgar Hızı:  2
Görüntüler İçeri Aktarıldı
SolarElevation:  1.1941756352352695
DOY:  160
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  3.669721800488686
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  301.37817
1. İterasyon Tamamlandı
Katsayı Değeri:  0.80939984
H1 Görüntüsü oluşturuldu
2. İterasyon Tamamlandı
Katsayı Değeri:  0.17895797
H2 Görüntüsü oluşturuldu
3. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.044996496
H3 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  300.16
Rüzgar Hızı:  2.4
Görüntüler İçeri Aktarıldı
SolarElevation:  1.24001337295249
DOY:  163
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  4.4036661

ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  300.31
Rüzgar Hızı:  2.7
Görüntüler İçeri Aktarıldı
SolarElevation:  0.854672513259162
DOY:  210
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  4.954124430659727
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  300.5054
1. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.11505969
H1 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  301.16
Rüzgar Hızı:  3
Görüntüler İçeri Aktarıldı
SolarElevation:  1.138132712632793
DOY:  212
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  5.504582700733029
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  300.353
1. İterasyon Tamamlandı
Katsayı Değeri:  inf
H1 Görüntüsü oluşturuldu
2. İterasyon Tamamlandı
Katsayı Değeri:  0.9995141
H2 Görüntüsü oluşturuldu
3. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.0
H3 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava 

H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  297.4153
1. İterasyon Tamamlandı
Katsayı Değeri:  inf
H1 Görüntüsü oluşturuldu
2. İterasyon Tamamlandı
Katsayı Değeri:  0.9995283
H2 Görüntüsü oluşturuldu
3. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.0
H3 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu
Hava Sıcaklığı:  299.16
Rüzgar Hızı:  3
Görüntüler İçeri Aktarıldı
SolarElevation:  0.5001086349333184
DOY:  268
Rn Görüntüsü Oluşturuldu
G Görüntüsü oluşturuldu
u100:  5.504582700733029
H ilk Görüntüsü Oluşturuldu
LST MASK Değeri:  296.25873
1. İterasyon Tamamlandı
Katsayı Değeri:  inf
H1 Görüntüsü oluşturuldu
2. İterasyon Tamamlandı
Katsayı Değeri:  0.99945873
H2 Görüntüsü oluşturuldu
3. İterasyonda Çözüm Bulundu
Katsayı Değeri:  0.0
H3 Görüntüsü oluşturuldu
LE Görüntüsü oluşturuldu
ETd Görüntüsü oluşturuldu
ETrF Görüntüsü oluşturuldu


# Klasörleme İşlemi

In [51]:
kayitYeri= r"D:\pytseb\uav-metric\METRIC\LE"
isExist = os.path.exists(kayitYeri)
if not isExist:
    os.makedirs(kayitYeri)
    print("Yeni Klasör Oluşturuldu!")
    
stack_= []

for root, dirname, files in os.walk(kayitYeri[:-2]):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-7:]=='_LE.tif':
            stack_.append(isim)
len(stack_)

Yeni Klasör Oluşturuldu!


37

In [52]:
for i in range(len(stack_)):
    #head, tail = os.path.split(stack_[i]) #dosya adını çekiyor
    shutil.move(stack_[i], kayitYeri )
print("Tamamlandı")

Tamamlandı


# Parsel Ortalamaları

In [29]:
klasor = r"D:\pytseb\uav-metric\METRIC\2020\ETd"
path= []


for root, dirname, files in os.walk(klasor):
    for x in files:
        isim=(root+'\\'+x)
        if isim[-13:]=='_ET_daily.tif':
            path.append(isim)
            
len(path)


shp=r"J:\2021_TUBITAK_sorgum\2020v2\shp\2020_sorgum_parseller.shp"    #2020
#shp=r"J:\2021_TUBITAK_sorgum\2021v2\ham\SHP\2021_sorgum_alfabetik.shp" #2021


head, tail = os.path.split(path[0]) #dosya adını çekiyor
tail[:]

kayitYeri= r"D:\pytseb\uav-metric"

for i in tqdm(range(0,len(path)), colour="#00ff00"):
    
    head, tail = os.path.split(path[i]) #dosya adını çekiyor    
    m=zonal_stats(shp, path[i] ,stats="mean")

    dframe=pd.DataFrame(m) 
    dframe.rename(columns={'mean':str(tail[:])}, inplace=True)

    dframe.to_csv(kayitYeri+'/'+tail[:8]+"_ETd.txt",sep="-",index=False) #index değerlerini kaldırmak için False

file_ndvi = glob.glob(os.path.join(kayitYeri+ "\*_ETd.txt"))
combined_ndvi_csv= pd.concat([pd.read_csv(f, sep="-") for f in file_ndvi], axis=1)
combined_ndvi_csv.index = combined_ndvi_csv.index + 1

writer = pd.ExcelWriter(kayitYeri+'/'+'2020-ET-METRIC-yeni.xlsx', engine='xlsxwriter')   #dosya adı
combined_ndvi_csv.to_excel(writer,sheet_name='2020-ET-METRIC')

writer.save()

for i in glob.glob(os.path.join(kayitYeri+"\*.txt")):
    os.remove(i)

print("Tamamlandı")  

100%|██████████████████████████████████████████| 37/37 [00:23<00:00,  1.59it/s]


Tamamlandı


In [96]:
klasor = r"J:\2021_TUBITAK_sorgum\2020v2\ham\saved\3BSI"
for root, dirname, files in os.walk(klasor):
    #print(root)
    if root[-6:]=="filtre":
        print(root.split("\\")[-2])
    

3BSI-EVI
3BSI-NDRE
3BSI-NDVI
3BSI-SAVI
3BSI-SR
